In [2]:
import re
_INDEX_EXTRACTOR = re.compile(r"([A-Za-z0-9_]+)\[?([0-9, ]*)\]?")

In [12]:
test = np.array(1)
test[()] = 2
test

array(2)

In [1]:
import copy
import re

import numpy as np
import dms_stan as dms
import dms_stan.model.components as dms_components

import itertools

ImportError: cannot import name 'AbstractParameter' from 'dms_stan.model.components.abstract_classes' (/home/bwittmann/GitRepos/DMSStan/dms_stan/model/components/abstract_classes.py)

First we define our model. For demo purposes, we will do an exponential growth model:

In [2]:
# Normalized time
time = np.linspace(0, 1, 5)

# Define the model
model = dms.model.ExponentialGrowthBinomialModel(
    t = time,
    counts = np.random.randint(0, 100, (1, 5, 100)),
    log_A = dms_components.Normal(mu=0.0, sigma=0.01, shape=(100,)),
    r = dms_components.Normal(mu=0.0, sigma=5.0, shape=(100,)),
    sigma = dms_components.HalfNormal(sigma=0.03, shape=())
)

In [3]:
stan_model = dms.model.components.stan.StanModel(model)

In [4]:
model.hyperparameters[0].child

In [5]:
model.hyperparameter_dict

{'sigma.mu': Hyperparameter(array(0.)),
 'sigma.sigma': Hyperparameter(array(0.03)),
 'log_A.mu': Hyperparameter(array(0.)),
 'log_A.sigma': Hyperparameter(array(0.01)),
 'r.mu': Hyperparameter(array(0.)),
 'r.sigma': Hyperparameter(array(5.))}

In [6]:
model.observables[0].model_varname

'counts'

In [7]:
# The number of levels is given by the dimensionality of the observable. We create
# lists for each level. Different variables will be defined and accessed depending
# on the level to which they belong. Note that we assume the last level is vectorized
n_levels = model.observables[0].stan_code_level
model_levels = [[] for _ in range(n_levels)]
transformed_data_levels = copy.deepcopy(model_levels)

# Get allowed index variable names for each level
allowed_index_names = tuple(
    char
    for char in dms.defaults.DEFAULT_INDEX_ORDER
    if {char, char.upper()}.isdisjoint(stan_model.all_varnames)
)

# There should be enough index names to cover all levels
if len(allowed_index_names) < (n_levels - 1):
    raise ValueError(
        f"Not enough index names ({len(allowed_index_names)}) to cover {n_levels} levels"
    )

# Observable is automatically the last level
model_levels[-1].append(model.observables[0].get_target_incrementation(allowed_index_names))

# # Loop over model parameters beginning from the observable
# for _, parent, _ in model.observables[0].recurse_parents():

    # If a named

# We are working up to the `get_stan_transformation` and `get_stan_distribution` methods

# Example:
model.counts.get_indexed_varname(allowed_index_names)

<class 'dms_stan.model.components.constants.Constant'>


ValueError: DMS Stan variable name not set. This is only set when the parameteris used in a DMS Stan model.

In [23]:
model.observables[0].parameters["N"].get_indexed_varname(allowed_index_names)

ValueError: DMS Stan variable name not set. This is only set when the parameteris used in a DMS Stan model.

In [19]:
model.observables[0].get_target_incrementation(allowed_index_names)

ValueError: DMS Stan variable name not set. This is only set when the parameteris used in a DMS Stan model.

In [8]:
model.counts.get_indexed_varname(allowed_index_names)

('counts[1,j,k]', True)